<a href="https://colab.research.google.com/github/hyweisky/gwings/blob/master/ExpressionsClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Defining an expression class
We define an abstract class Expr, representing a generic expression. This generic class has as subclasses the classes that represent the various operators, such as Plus, Minus, Multiply, as well as the variable class V. Here is our basic implementation.

In [ ]:
class Expr(object):
    """Abstract class representing expressions"""

    name = "expr" # Not used, but just to define it.

    def __init__(self, *args):
        """An object is created by passing to the constructor the children"""
        self.children = args
        self.value = None # The value of the expression
        self.child_values = None # The values of the children; useful to have

    def eval(self):
        """Evaluates the expression."""
        # First, we evaluate the children.
        self.child_values = [c.eval() if isinstance(c, Expr) else c
                             for c in self.children]
        # Then, we evaluate the expression itself.
        self.value = self.op(*self.child_values)
        return self.value

    def op(self):
        """This operator must be implemented in subclasses; it should
        compute self.value from self.values, thus implementing the
        operator at the expression node."""
        raise NotImplementedError()

    def __repr__(self):
        """Represents the expression in a somewhat readable way."""
        if len(self.children) == 1:
            # Unary operators
            return "({}{})".format(self.__class__.name, self.children[0])
        elif len(self.children) == 2:
            return "({} {} {})".format(
                self.children[0], self.__class__.name, self.children[1]
            )
        # Catch-all.
        return "{}({})".format(self.__class__.__name__,
                                ', '.join(repr(c) for c in self.children))

    # Expression constructors

    def __add__(self, other):
        return Plus(self, other)

    def __radd__(self, other):
        return Plus(self, other)

    def __sub__(self, other):
        return Minus(self, other)

    def __rsub__(self, other):
        return Minus(other, self)

    def __mul__(self, other):
        return Multiply(self, other)

    def __rmul__(self, other):
        return Multiply(other, self)

    def __truediv__(self, other):
        return Divide(self, other)

    def __rtruediv__(self, other):
        return Divide(other, self)

    def __neg__(self):
        return Negative(self)




Variables are created specifying a name, and an initial value. If nothing is specified, variables have random initial values. You can assign a value to a variable using the assign method. The eval method of a variable simply returns its value.





In [ ]:
import random
import string

class V(Expr):
    """Variable."""

    def __init__(self, value=None):
        super().__init__()
        self.children = []
        self.value = random.gauss(0, 1) if value is None else value
        self.name = ''.join(
            random.choices(string.ascii_letters + string.digits, k=3))

    def eval(self):
        return self.value

    def assign(self, value):
        self.value = value

    def __repr__(self):
        return "V({}, value={})".format(self.name, self.value)

Here are the constructors for the other operators; for them, we just need to provide an implementation for op, since all the rest is inherited from Expr. We actually also define a name, as a class attribute, that we use in the representation method.

In [ ]:
class Plus(Expr):
    name = "+"
    def op(self, x, y):
        return x + y

class Minus(Expr):
    name = "-"
    def op(self, x, y):
        return x - y

class Multiply(Expr):
    name = "*"
    def op(self, x, y):
        return x * y

class Divide(Expr):
    name = "/"
    def op(self, x, y):
        return x / y

class Negative(Expr):
    name = "-"
    def op(self, x):
        return -x

# We can build and evaluate expressions quite simply.

In [ ]:
e = V(2) + 3
print(type(e))
print(e.eval())

e = (V() + V(2)) * (2 + V(1))
print(e)
print(e.eval())

<class '__main__.Plus'>
5
((V(9hp, value=-0.34519492056815754) + V(FUp, value=2)) * (V(7tx, value=1) + 2))
4.964415238295528


If we want to be able to assign values to variables, or refer to them in our code later, we need to assign our variable objects to Python variables:

In [ ]:
x = V()
y = V()
e = x + y
print(e.eval()) # This uses the initial random values.

x.assign(2)
y.assign(3)
print(e.eval())


-1.655549907584903
5


# Defining Expression Equality
If we test equality between expressions, we are in for a surprise.

In [ ]:
x = V()
e1 = x + 4
e2 = x + 4
e1 == e2

False

# Why is the result False?

Python knows how to compare objects that belong to its own types. So you can do comparisons between strings, numbers, tuples, and more, and it all works as expected. This is why we could check equality of expressions represented as trees: those expression trees are composed entirely of standard Python types, namely, strings, numbers, and tuples.

However, Expr, V, etc, are classes we defined, and Python has no idea of what it means for objects of user-defined classes to be equal.
In this case, Python defaults to considering equal two objects if they are the same object. The two expressions e1 and e2 above are not the same object: they are two distinct objects, which just happen to represent the same expression.

If we want to have a notion of expression equality that represents our idea that "two expression objects are equal if they represent the same expression", we need to define equality ourselves. This can be easily done, by defining an _ _ eq _ _ method. This method has the form:

def __eq__(self, other):
    ...
    return <True/False>

Here, self is the object on which the method is called, and other is another object -- any other object. Our job is to define when the object self is equal to the object other. This can be easily done; using again our way of adding methods to existing classes, we write:

In [ ]:
def expr_eq(self, other):
    if isinstance(other, Expr):
        # The operators have to be the same
        if self.__class__ != other.__class__:
            return False
        # and their corresponding children need to be equal
        if len(self.children) != len(other.children):
            return False
        for c1, c2 in zip(self.children, other.children):
            if c1 != c2: return False
        return True
    else:
        return False

Expr.__eq__ = expr_eq

If we did not define equality for variables, two variables would be considered equal according to Expr.__eq__, since V is a subclass of Expr. This would yield non-intended consequences (all variables would be considered equal).

Thus, we define equality for variables to be the basic equality for objects: two variables are equal iff they are the same object. In the definition below, object.__eq__ is this primitive notion of equality, defined over the class object of Python, which is the base class for all classes.

In [ ]:
#Test object.eq
V.__eq__  = object.__eq__
x = V()
y = V()
z = x
print(x == y)
print(x == z)

False
True


# Once expression equality is thus defined, we get the expected result when we compare expressions:

In [ ]:
x = V()
e1 = x + 4
e2 = x + 4
e1 == e2

True

Having to define equality "by hand" is very pedantic, but it does give us the flexibility of defining precisely what it means for two expressions to be equal.

Variable Occurrence
Now that we have expressions, let us play with them. First, as a warm-up exercise, let us write an occurs method for expressions, which checks if a given variable occurs in the expression. First we define it for a variable: of course, a variable occurs in itself only if the variable is the same as the one whose occurrence we are checking.

# Question 1: Variable occurrence in variables

In [ ]:
### Variable occurrence in a variable

def v_contains(self, var):
    """Returns True of var is the same as self, and False otherwise."""
    ### YOUR CODE HERE
    return var==self

V.__contains__ = v_contains


In [ ]:
## Here you can also test your code. 

x = V()
y = V()
print(x in x)
print(x in y)


### Tests for variable occurrence

x = V()
y = V()
assert x in x
assert not x in y
z = x
assert x in z

True
False


# Question 2: Occurrence of a variable in an expression
Once we define occurrence of a variable in a variable, we can define occurrence in a variable in a general expression. Of course, a variable appears in an expression if it appears in some of its children.

In [ ]:
### Occurrence of a variable in an expression

def expr_contains(self, var):
    ### YOUR CODE HERE
    contains=False
    if isinstance(self, V):
      return self==var
    for v in self.children:
      if isinstance(v, Expr):
        contains=contains | expr_contains(v,var)
      elif v==var:contains=True
    return contains

Expr.__contains__ = expr_contains

In [ ]:
## Here you can also test your code. 

x = V()
y = V()
z = V()
e = x+(2+y)

print(x in e)
print(y in e)
print(z in e)

True
True
False


In [ ]:
## Tests for occurrence: 5 points. 

x = V()
y = V()
z = V()
e = x + (2 * y)

assert x in e
assert y in e
assert z not in e


## Hidden tests for occurrence: 5 points. 

# Variable Substitution
Another fun thing we can do is substitute a variable with an expression. Suppose you define an expression:

x = V()

y = V()

e = (x + 1) * (y + 1)
#Suppose you also have another expression:

z = V()

f = y + z
#Then, you can replace all occurrences of variable x in e with expression f:

new_e = e.replace(x, f)
#and new_e should be then equal to:

((y + z) + 1) * (y + 1)

Let us implement variable substitution. Let us begin by defining variable substitution for variables.


# Question 3: Variable replacement for variables

In [ ]:
### Variable replacement in variables

def v_replace(self, x, e):
    """If self is x, replaces all occurrences of x with e."""
    ### YOUR CODE HERE
    if self==x:
      self=e
    return self
V.replace = v_replace

In [ ]:
## Here you can also test your code

x = V()
y = V()
z = V()
print(x.replace(x, x+y))
print(x == x.replace(x, x))
print(y == x.replace(x, y))
print(x == x.replace(y, z))

(V(T0j, value=0.11098013540054927) + V(iNh, value=0.2901431232758615))
True
True
True


In [ ]:
## Tests for variable replacement in variables. 2 points. 
x = V()
y = V()
z = V()
assert x == x.replace(x, x)
assert y == x.replace(x, y)
assert x == x.replace(y, z)
assert x.replace(x, y).replace(y, z) == z

In [ ]:
## Other tests for variable replacement. 2 points. 

x = V()
y = V()

e = x.replace(x, y + 3 * x)

x.assign(2)
y.assign(3)
assert e.eval() == 9
## Hidden tests for variable replacement.  2 points. 

We now define variable replacement for expressions. Consider a simple expression:

x = V()

y = V()

z = V()

e = x + y

Suppose we want to return e.replace(x, z). The idea is:

carry out the replacement for each child (via a recursive call, as usual),
then return an expression built out of the replacements.
For instance, consider e = x + y. To compute

e.replace(x, z)
we first replace x with z, and then we return Plus(z, y).

The problem is exactly in the last sentence. A Plus object, after carrying out the replacement in the children, should return a Plus object with the new children. Similarly, a Minus object should return a Minus object, a Multiply object should return a Multiply object, and so forth. If we implement this in the straightforward way, we need to add a replace method to all of these classes, so that they can return an object of the appropriate type. This is a lot of work.

Is there a better way? It turns out, yes. In an object,

self.__class__
is the class of the object. So if you want to return a new object of the same class, created say with arguments x and y, all you need to do is:

self.__class__(x, y)
In this way, if you are in a Plus object, self.__class__ is Plus, and everything works. Using this idea, we, that is, you, can implement the replacement method directly for the Expr class.

# Question 4: Replacement for expressions

In [ ]:
### Replacement for expressions

def expr_replace(self, x, e):
    ### YOUR CODE HERE
    childs=[]
    if x in self:
      for c in self.children:
        if isinstance(c,V) or isinstance(c,Expr):     
          childs.append(c.replace(x,e))
        else:
          childs.append(c)
    else:
      return self
    return self.__class__(*childs)
Expr.replace = expr_replace


In [ ]:
### Here you can debug your code. 

x = V()
y = V()
z = V()

e = (1 + x) * (1 + y)
x.assign(1)
y.assign(2)
z.assign(3)
# e = x * (y+1)
print(x,y,z,e.eval())
f = e.replace(x, x + z)

print(f.eval())



V(L6F, value=1) V(S4t, value=2) V(kHe, value=3) 6
15


In [ ]:
### Tests for expression replacement. 7 points. 

x = V()
y = V()
z = V()

e = (1 + x) * (1 + y)
f = e.replace(x, x + z)
assert f == (1 + (x + z)) * (1 + y)

x.assign(1)
y.assign(2)
z.assign(3)
assert e.eval() == 6
assert f.eval() == 15

e = (x + y) / (x - y)
f = e.replace(x, 2 * x).replace(y, 3 * y)
assert f.eval() == (2 + 6) / (2 - 6)

### Hidden tests for expression replacement. 8 points.

Expression Derivation
We will develop here a method derivate such that, for an expression e, the method call e.derivate(x) returns the derivative of the expression with respect to the variable x. As in the previous chapter, we use the following derivation formulas:#(具体见word)
# Question 5. Derivation for a variable

In [ ]:
### Derivation of variables

def v_derivate(self, x):
    ### YOUR CODE HERE
   return 1 if self==x else 0

V.derivate = v_derivate


In [ ]:

## Here you can debug your code. 

x = V()
y = V()
print(x.derivate(x))
print(x.derivate(y))

1
0


In [ ]:
## Tests for variable derivation

x = V()
y = V()
assert x.derivate(x) == 1
assert x.derivate(y) == 0

# Derivation for expressions
This time, there is no clever trick to implement derivate as a method of Expr, because the derivative behaves in a different way for the different operators. Hence, we will need to implement derivate for each individual operator. We let you do it. There are two things to be careful about:

Children of an operator may not be expressions; they can also be constants such as 2.3 or 4.1.
If the new children of an expression are all numbers, return the numerical result of the expression rather than a symbolic expression. For example, do not return Plus(1, 0); rather, just return 1. Note that, if you put the derivatives of the two children in, say, df and dg, you can simplify automatically by returning df + dg, which will return an expression if one of df or dg is an expression, and a number otherwise.
We give you the solution for Plus, mainly because it might be difficult to believe that the solution is this simple.

In [ ]:
def plus_derivate(self, x):
    f, g = self.children
    df = f.derivate(x) if isinstance(f, Expr) else 0
    dg = g.derivate(x) if isinstance(g, Expr) else 0
    return df + dg
Plus.derivate = plus_derivate

# Question 6: Derivation for Minus
You can now work out the case for Minus, which is very similar.

In [ ]:
### Derivative of Minus

def minus_derivate(self, x):
    ### YOUR CODE HERE
    f, g = self.children
    df = f.derivate(x) if isinstance(f, Expr) else 0
    dg = g.derivate(x) if isinstance(g, Expr) else 0
    return df - dg
Minus.derivate = minus_derivate

In [ ]:
### Here you can debug your code. 

x = V()
y = V()
z = V()

e =x - y
e.derivate(y)


-1

In [ ]:
### Tests for derivatives of Plus and Minus. 3 points. 
x = V()
y = V()
z = V()
e = x + y
assert e.derivate(x) == 1
f = x - y
assert f.derivate(x) == 1
assert f.derivate(y) == -1

h = e + f
assert h.derivate(x) == 2

u = x + 4
assert u.derivate(x) == 1
v = 3 - y
assert v.derivate(x) == 0
assert v.derivate(y) == -1

# Question 7: derivative of Negative
Since we are at it, let us take care of unary minus, or Negative.

In [ ]:
### Derivative of Negative

def negative_derivate(self, x):
    ### YOUR CODE HERE
    f,= self.children
    df = f.derivate(x) if isinstance(f, Expr) else 0
    return -df
Negative.derivate = negative_derivate


In [ ]:
### Here you can debug your code.

x = V()
y = V()
e = x-(-y)
e.derivate(y)


1

In [ ]:
### Tests for negative. 4 points. 

x = V()
y = V()
e = -x
assert e.derivate(x) == -1
assert e.derivate(y) == 0

In [ ]:

### Tests for negative. 4 points. 
x = V()
y = V()
e = -x
print(e.derivate(x))
assert e.derivate(x) == -1
assert e.derivate(y) == 0

-1


Now for multiplication and divisions. Be careful to use the formulas exactly as given, otherwise the result may not match. E.g. use#(具体见word)

# Question 8: Derivation of multiplication

In [ ]:
### Derivative of multiplication.
def multiply_derivate(self, x):
    ### YOUR CODE HERE
    f,g=self.children
    df = f.derivate(x) if isinstance(f, Expr) else 0
    dg = g.derivate(x) if isinstance(g, Expr) else 0
    return df*g+f*dg

Multiply.derivate = multiply_derivate

If you did the above right, it will be 4-5 lines wrong. If you wrote much more than that, please think at it again. The solution is quite simple; you can just trust that the operators * and + will build it.

In [ ]:
## Here you can debug your code. 

x = V()
e = x * x
de = e.derivate(x)
de


((1 * V(TP9, value=-0.5411262336133215)) + (V(TP9, value=-0.5411262336133215) * 1))

In [ ]:
## Tests for derivative of multiplication. 4 points. 

x = V()
y = V(value=2)

e = x * y
# This is ugly.  It is.  We have not implemented 0, 1 simplifications.
print(e.derivate(x))
assert e.derivate(x) == 1 * y + x * 0

((1 * V(Y7p, value=2)) + (V(bNr, value=-2.4503790300907107) * 0))


In [ ]:
# To remedy ugliness, we now test numerically.
f = x * x
x.assign(3)
assert f.derivate(x).eval() == 6, f.derivate(x).eval()
x.assign(4)
assert f.derivate(x).eval() == 8, f.derivate(x).eval()

h = 3 * x
assert h.derivate(x).eval() == 3
u = x * 3
assert u.derivate(x).eval() == 3

## Question 9: Derivative of division#(具体见word)

In [ ]:
### Derivative of division

def divide_derivate(self, x):
    ### YOUR CODE HERE
    f,g=self.children
    df = f.derivate(x) if isinstance(f, Expr) else 0
    dg = g.derivate(x) if isinstance(g, Expr) else 0
    return (df*g-f*dg)/(g*g)
Divide.derivate = divide_derivate

In [ ]:
## Here you can debug your code. 

x = V()
y = V()
print("x:", x)
print("y:", y)

e = x / y

e.derivate(y)


x: V(H2a, value=0.9142714871773974)
y: V(fOu, value=-0.8977765171991245)


(((0 * V(fOu, value=-0.8977765171991245)) - (V(H2a, value=0.9142714871773974) * 1)) / (V(fOu, value=-0.8977765171991245) * V(fOu, value=-0.8977765171991245)))

In [ ]:

### Tests for derivative of division. 4 points. 

x = V()
y = V()
e = x / 2
f = e.derivate(x)
print('f=',f)
x.assign(3)
assert f.eval() == 1/2

g = 1 / x
assert g.derivate(x).eval() == - 1 / 9
assert g.derivate(y).eval() == 0


f= ((2 - (V(3kH, value=0.15408337313733367) * 0)) / 4)


In [ ]:
## Miscellaneous tests.  3 points. 

x = V(value=2)
y = V(value=3)

f = (x + 1) * (y + 1) / (x - 1) * (y - 1)
df = f.derivate(x)

assert df.eval() == -16

x.assign(0.5)
y.assign(0.5)
assert df.eval() == 6

In [ ]:
## Miscellaneous tests. 3 points. 

x = V(value=0)

f = (3 * x * x + 2 * x + 4) / (5 * x * x - 7 * x + 12)

df = f.derivate(x)
assert abs(df.eval() - 0.3611) < 0.01
x.assign(1)
assert df.eval() == 0.53
x.assign(1000)
assert abs(df.eval()) < 0.001
x.assign(-0.5)
assert abs(df.eval() - 0.1) < 0.01

## Final hidden tests on everything. 10 points. 